![Add a relevant banner image here](path_to_image)

# Project Title

## Overview

Short project description. Your bottom line up front (BLUF) insights.

## Business Understanding

Text here

## Data Understanding

Text here

In [1]:
# Load relevant imports here
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import gc

In [2]:
df_all_data = pd.read_csv('Data/US_Accidents_March23.csv')
print(df_all_data.describe())
print(df_all_data.head())
print(df_all_data.columns)

           Severity     Start_Lat     Start_Lng       End_Lat       End_Lng  \
count  7.728394e+06  7.728394e+06  7.728394e+06  4.325632e+06  4.325632e+06   
mean   2.212384e+00  3.620119e+01 -9.470255e+01  3.626183e+01 -9.572557e+01   
std    4.875313e-01  5.076079e+00  1.739176e+01  5.272905e+00  1.810793e+01   
min    1.000000e+00  2.455480e+01 -1.246238e+02  2.456601e+01 -1.245457e+02   
25%    2.000000e+00  3.339963e+01 -1.172194e+02  3.346207e+01 -1.177543e+02   
50%    2.000000e+00  3.582397e+01 -8.776662e+01  3.618349e+01 -8.802789e+01   
75%    2.000000e+00  4.008496e+01 -8.035368e+01  4.017892e+01 -8.024709e+01   
max    4.000000e+00  4.900220e+01 -6.711317e+01  4.907500e+01 -6.710924e+01   

       Distance(mi)  Temperature(F)  Wind_Chill(F)   Humidity(%)  \
count  7.728394e+06    7.564541e+06   5.729375e+06  7.554250e+06   
mean   5.618423e-01    6.166329e+01   5.825105e+01  6.483104e+01   
std    1.776811e+00    1.901365e+01   2.238983e+01  2.282097e+01   
min    0.000000e

In [ ]:
df_severity_by_state = pd.crosstab(df_all_data['Severity'], df_all_data['State'])
df_severity_by_state.head()

In [ ]:
# chart of accident severity by state
states = df_severity_by_state.columns
sev1 = df_severity_by_state.iloc[0]
sev2 = df_severity_by_state.iloc[1]
sev3 = df_severity_by_state.iloc[2]
sev4 = ct_sev4_by_state = df_severity_by_state.iloc[3]

plt.figure(figsize=(12, 7))
plt.bar(states, sev1)
plt.bar(states, sev2, bottom=sev1)
plt.bar(states, sev3, bottom=sev1+sev2)
plt.bar(states, sev4, bottom=sev1+sev2+sev3)
plt.xlabel("State")
plt.xticks(rotation=60)
plt.ylabel("Number of Accidents")
plt.legend(["Severity 1", "Severity 2", "Severity 3", "Severity 4"])
plt.title("Accident Severity by State (2016 - 2023)")
plt.show()

In [ ]:
# chart of accident severity by state without sev2 (sev 2 >> than the others so obscures sev 1, 3, 4 above)
states = df_severity_by_state.columns
sev1 = df_severity_by_state.iloc[0]
sev3 = df_severity_by_state.iloc[2]
sev4 = ct_sev4_by_state = df_severity_by_state.iloc[3]

plt.figure(figsize=(12, 7))
plt.bar(states, sev1)
plt.bar(states, sev3)
plt.bar(states, sev4)
plt.xlabel("State")
plt.xticks(rotation=60)
plt.ylabel("Number of Accidents")
plt.legend(["Severity 1", "Severity 3", "Severity 4"])
plt.title("Accident Severity by State, Excluding Severity 2 (2016 - 2023)")
plt.show()

In [ ]:
df_sev_by_crossing = pd.crosstab(df_all_data['Severity'], df_all_data['Crossing'])
df_sev_by_crossing = df_sev_by_crossing.rename(columns={False: "No", True: "Yes"})
df_sev_by_crossing.head()

In [ ]:
# charts of accident severity by bump, traffic calming, roundabout
# chart of accident severity by state
crossing = df_sev_by_crossing.columns
cr1 = df_sev_by_crossing.iloc[0]
cr2 = df_sev_by_crossing.iloc[1]
cr3 = df_sev_by_crossing.iloc[2]
cr4 = df_sev_by_crossing.iloc[3]

plt.figure(figsize=(10, 7))
plt.bar(crossing, cr1)
plt.bar(crossing, cr2)
plt.bar(crossing, cr3)
plt.bar(crossing, cr4)
plt.xticks(rotation=60)
plt.xlabel("Nearby Crossing")
plt.ylabel("Number of Accidents")
plt.legend(["Severity 1", "Severity 2", "Severity 3", "Severity 4"])
plt.title("Accident Severity by Proximity to Crossing, (2016 - 2023)")
plt.show()

In [ ]:
print(f"Count of Wind Direction Entries: {df_all_data['Wind_Direction'].nunique()}")


## Data Preparation

### Data Selection

Based on my exploration of the data, I'm dropping the following fields from the dataset for the following reasons:

- Source: contains information that has no relationship to causes and effects of accidents
- Timezone: duplicates zip/state with less precision
- Country: all data is from the United States so this field is redundant
- Airport_Code: doesn't provide germane information-the exact location where weather conditions are reported is not a variable that can be adjusted
- Weather_Timestamp: not related to the conditions of the accidents in any way
- Wind_Direction: too many unique values; values are also not related to travel directions so it's unlikely they'll  produce clear/actionable conclusions


In [3]:
df_refined = df_all_data.drop(['Source', 'Timezone', 'Country', 'Airport_Code', 'Weather_Timestamp', 'Wind_Direction'], axis=1)

print(df_refined.describe())
print(df_refined.info())
print(df_refined.columns)

           Severity     Start_Lat     Start_Lng       End_Lat       End_Lng  \
count  7.728394e+06  7.728394e+06  7.728394e+06  4.325632e+06  4.325632e+06   
mean   2.212384e+00  3.620119e+01 -9.470255e+01  3.626183e+01 -9.572557e+01   
std    4.875313e-01  5.076079e+00  1.739176e+01  5.272905e+00  1.810793e+01   
min    1.000000e+00  2.455480e+01 -1.246238e+02  2.456601e+01 -1.245457e+02   
25%    2.000000e+00  3.339963e+01 -1.172194e+02  3.346207e+01 -1.177543e+02   
50%    2.000000e+00  3.582397e+01 -8.776662e+01  3.618349e+01 -8.802789e+01   
75%    2.000000e+00  4.008496e+01 -8.035368e+01  4.017892e+01 -8.024709e+01   
max    4.000000e+00  4.900220e+01 -6.711317e+01  4.907500e+01 -6.710924e+01   

       Distance(mi)  Temperature(F)  Wind_Chill(F)   Humidity(%)  \
count  7.728394e+06    7.564541e+06   5.729375e+06  7.554250e+06   
mean   5.618423e-01    6.166329e+01   5.825105e+01  6.483104e+01   
std    1.776811e+00    1.901365e+01   2.238983e+01  2.282097e+01   
min    0.000000e

### Data Cleaning

##### Missing Values

I've managed missing values in the code blocks below. Here's a brief explanation of my approch to each column:

- Temperature, Wind_Chill, Humidity, Pressure, Visibility: imputed based on the mean temp of other accident entries sharing the same day and zip code (or state if there are none in the zip code)
- Precipitation, Wind_Speed: assumed NaN indicates no precipitation/wind and replaced NaN with zero
- Weather_Condition: consolidated entries from 144 to 30
- Sunrise_Sunset: imputed based on
- Civil_Twilight: imputed based on
- Nautical_Twilight: imputed based on
- Astronomical_Twilight: imputed based on

In [14]:
df_refined['Start_Time'] = pd.to_datetime(df_refined['Start_Time'], yearfirst=True, format='mixed')
df_refined['End_Time'] = pd.to_datetime(df_refined['End_Time'], yearfirst=True, format='mixed')
df_refined['Acc_date'] = df_refined['Start_Time'].dt.date
df_refined.tail(5)

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Acc_date
7728389,A-7777757,2,2019-08-23 18:03:25,2019-08-23 18:32:01,34.00248,-117.37936,33.99888,-117.37094,0.543,At Market St - Accident.,...,False,False,False,False,False,Day,Day,Day,Day,2019-08-23
7728390,A-7777758,2,2019-08-23 19:11:30,2019-08-23 19:38:23,32.76696,-117.14806,32.76555,-117.15363,0.338,At Camino Del Rio/Mission Center Rd - Accident.,...,False,False,False,False,False,Day,Day,Day,Day,2019-08-23
7728391,A-7777759,2,2019-08-23 19:00:21,2019-08-23 19:28:49,33.77545,-117.84779,33.77740,-117.85727,0.561,At Glassell St/Grand Ave - Accident. in the ri...,...,False,False,False,False,False,Day,Day,Day,Day,2019-08-23
7728392,A-7777760,2,2019-08-23 19:00:21,2019-08-23 19:29:42,33.99246,-118.40302,33.98311,-118.39565,0.772,At CA-90/Marina Fwy/Jefferson Blvd - Accident.,...,False,False,False,False,False,Day,Day,Day,Day,2019-08-23
7728393,A-7777761,2,2019-08-23 18:52:06,2019-08-23 19:21:31,34.13393,-117.23092,34.13736,-117.23934,0.537,At Highland Ave/Arden Ave - Accident.,...,False,False,False,False,False,Day,Day,Day,Day,2019-08-23


In [15]:
def impute_missing_data(input_df, date='Acc_date', zip='Zipcode', state='State', temp='Temperature(F)', 
                        windchill='Wind_Chill(F)', hum='Humidity(%)', press='Pressure(in)', vis='Visibility(mi)'):
    """
    Replaces NaN values with the mean values of entries
    with the same date and zip code. If no match exists with date and county,
    uses date and state.
    """

    def fill_missing_temperatures(input_df, date='date', zip='zip', state='state', temp='temp'):
        df_filled = df_refined.copy()
        
        # Step 1: mean temperature for each date-zip combination
        temp_means_zip = df_filled.groupby([date, zip])[temp].transform('mean')
        
        # fill NaN values with the date-zip group mean
        df_filled[temp] = df_filled[temp].fillna(temp_means_zip)
        
        # Step 2: date-state combination for remaining NaNs
        if df_filled[temp].isna().any():
            remaining = df_filled[temp].isna().sum()
            print(f"Info: {remaining} temperatures still missing after date-zip fill.")
            print("Filling remaining with date-state mean.")
            
            temp_means_state = df_filled.groupby([date, state])[temp].transform('mean')
            df_filled[temp] = df_filled[temp].fillna(temp_means_state)
        
        # Step 3: if any NaNs still remain, fill with overall mean as last resort
        if df_filled[temp].isna().any():
            remaining = df_filled[temp].isna().sum()
            print(f"Warning: {remaining} temperatures still missing after date-state fill.")
            print("Filling remaining with overall mean as last resort.")
            df_filled[temp] = df_filled[temp].fillna(df_filled[temp].mean())
        
        return df_filled

    def fill_missing_windchill(df='df_filled', date='date', zip='zip', state='state', windchill='windchill'):
        # Step 1: mean windchill for each date-zip combination
        windchill_means_zip = df_filled.groupby([date, zip])[windchill].transform('mean')
        
        # fill NaN values with the date-zip group mean
        df_filled[windchill] = df_filled[windchill].fillna(windchill_means_zip)
        
        # Step 2: date-state combination for remaining NaNs
        if df_filled[windchill].isna().any():
            remaining = df_filled[windchill].isna().sum()
            print(f"Info: {remaining} wind chills still missing after date-zip fill.")
            print("Filling remaining with date-state mean.")
            
            windchill_means_state = df_filled.groupby([date, state])[windchill].transform('mean')
            df_filled[windchill] = df_filled[windchill].fillna(windchill_means_state)
        
        # Step 3: if any NaNs still remain, fill with overall mean as last resort
        if df_filled[windchill].isna().any():
            remaining = df_filled[windchill].isna().sum()
            print(f"Warning: {remaining} wind chills still missing after date-state fill.")
            print("Filling remaining with overall mean as last resort.")
            df_filled[windchill] = df_filled[windchill].fillna(df_filled[windchill].mean())
        
        return df_filled
    
    def fill_missing_press(df='df_filled', date='date', zip='zip', state='state', press='press'):
        # Step 1: mean pressure for each date-zip combination
        press_means_zip = df_filled.groupby([date, zip])[press].transform('mean')
        
        # fill NaN values with the date-zip group mean
        df_filled[press] = df_filled[press].fillna(press_means_zip)
        
        # Step 2: date-state combination for remaining NaNs
        if df_filled[press].isna().any():
            remaining = df_filled[press].isna().sum()
            print(f"Info: {remaining} pressure values still missing after date-zip fill.")
            print("Filling remaining with date-state mean.")
            
            press_means_state = df_filled.groupby([date, state])[press].transform('mean')
            df_filled[press] = df_filled[press].fillna(hum_means_state)
        
        # Step 3: if any NaNs still remain, fill with overall mean as last resort
        if df_filled[press].isna().any():
            remaining = df_filled[press].isna().sum()
            print(f"Warning: {remaining} pressure values still missing after date-state fill.")
            print("Filling remaining with overall mean as last resort.")
            df_filled[press] = df_filled[press].fillna(df_filled[press].mean())
        
        return df_filled
    
    def fill_missing_vis(df='df_filled', date='date', zip='zip', state='state', vis='vis'):
        # Step 1: mean pressure for each date-zip combination
        vis_means_zip = df_filled.groupby([date, zip])[vis].transform('mean')
        
        # fill NaN values with the date-zip group mean
        df_filled[vis] = df_filled[vis].fillna(vis_means_zip)
        
        # Step 2: date-state combination for remaining NaNs
        if df_filled[vis].isna().any():
            remaining = df_filled[vis].isna().sum()
            print(f"Info: {remaining} visibility values still missing after date-zip fill.")
            print("Filling remaining with date-state mean.")
            
            vis_means_state = df_filled.groupby([date, state])[vis].transform('mean')
            df_filled[vis] = df_filled[vis].fillna(vis_means_state)
        
        # Step 3: if any NaNs still remain, fill with overall mean as last resort
        if df_filled[vis].isna().any():
            remaining = df_filled[vis].isna().sum()
            print(f"Warning: {remaining} visibility values still missing after date-state fill.")
            print("Filling remaining with overall mean as last resort.")
            df_filled[vis] = df_filled[vis].fillna(df_filled[vis].mean())
        
        return df_filled
    
    return df_filled
    

df_filled = impute_missing_data(df_refined)

In [ ]:
# replacing NaN values in the temperature column

def fill_missing_temperatures(df, date_col='Acc_date', zip_col='Zipcode', state_col='State', temp_col='Temperature(F)'):
    """
    Replaces NaN Temperature values with the mean temperature of entries
    with the same date and zip code. If no match exists with date and county,
    uses date and state.
    """
    # Step 1: mean temperature for each date-zip combination
    df_temp_fill = df_refined.copy()
    
    temp_means_zip = df_temp_fill.groupby([date_col, zip_col])[temp_col].transform('mean')
    
    # fill NaN values with the date-zip group mean
    df_temp_fill[temp_col] = df_temp_fill[temp_col].fillna(temp_means_zip)
    
    # Step 2: date-state combination for remaining NaNs
    if df_temp_fill[temp_col].isna().any():
        remaining = df_temp_fill[temp_col].isna().sum()
        print(f"Info: {remaining} temperatures still missing after date-zip fill.")
        print("Filling remaining with date-state mean.")
        
        temp_means_state = df_temp_fill.groupby([date_col, state_col])[temp_col].transform('mean')
        df_temp_fill[temp_col] = df_temp_fill[temp_col].fillna(temp_means_state)
    
    # Step 3: if any NaNs still remain, fill with overall mean as last resort
    if df_temp_fill[temp_col].isna().any():
        remaining = df_temp_fill[temp_col].isna().sum()
        print(f"Warning: {remaining} temperatures still missing after date-state fill.")
        print("Filling remaining with overall mean as last resort.")
        df_temp_fill[temp_col] = df_temp_fill[temp_col].fillna(df_temp_fill[temp_col].mean())
    
    return df_temp_fill



df_temp_fill = fill_missing_temperatures(df_refined)

In [17]:
# replacing NaN values in the temperature column

def fill_missing_temp(df, date='Acc_date', zip='Zipcode', state='State', temp='Temperature(F)'):
    """
    Replaces NaN humidity values with the mean humidity of entries
    with the same date and zip code. If no match exists with date and county,
    uses date and state.
    """
    # Step 1: mean humidity for each date-zip combination
    df_temp_fill = df_refined.copy()
    
    temp_means_zip = df_temp_fill.groupby([date, zip])[temp].transform('mean')
    
    # fill NaN values with the date-zip group mean
    df_temp_fill[temp] = df_temp_fill[temp].fillna(temp_means_zip)
    
    # Step 2: date-state combination for remaining NaNs
    if df_temp_fill[temp].isna().any():
        remaining = df_temp_fill[temp].isna().sum()
        print(f"Info: {remaining} temperature entries still missing after date-zip fill.")
        print("Filling remaining with date-state mean.")
        
        temp_means_state = df_temp_fill.groupby([date, state])[temp].transform('mean')
        df_temp_fill[temp] = df_temp_fill[temp].fillna(temp_means_state)
    
    # Step 3: if any NaNs still remain, fill with overall mean as last resort
    if df_temp_fill[temp].isna().any():
        remaining = df_temp_fill[temp].isna().sum()
        print(f"Warning: {remaining} temperature entries still missing after date-state fill.")
        print("Filling remaining with overall mean as last resort.")
        df_temp_fill[temp] = df_temp_fill[temp].fillna(df_temp_fill[temp].mean())
    
    return df_temp_fill

df_temp_fill = fill_missing_temp(df_refined)

Info: 158386 temperature entries still missing after date-zip fill.
Filling remaining with date-state mean.
Filling remaining with overall mean as last resort.


In [18]:
# verifying NaNs have been replaced
df_temp_fill['Temperature(F)'].isna().sum()

0

In [20]:
# writing NaN-free column back to the original dataframe
df_refined['Temperature(F)'] = df_temp_fill['Temperature(F)']
print(f"Entries with NaN Temperature: {df_refined['Temperature(F)'].isna().sum()}")

Entries with NaN Temperature: 0


In [21]:
# deleting working df to free up memory
del df_temp_fill
gc.collect()

1031

In [22]:
# replacing NaN values in the wind chill column

def fill_missing_windchill(df, date='Acc_date', zip='Zipcode', state='State', windchill='Wind_Chill(F)'):
    """
    Replaces NaN wind chill values with the mean wind chill of entries
    with the same date and zip code. If no match exists with date and county,
    uses date and state.
    """
    # Step 1: mean temperature for each date-zip combination
    df_windchill_fill = df_refined.copy()
    
    windchill_means_zip = df_windchill_fill.groupby([date, zip])[windchill].transform('mean')
    
    # fill NaN values with the date-zip group mean
    df_windchill_fill[windchill] = df_windchill_fill[windchill].fillna(windchill_means_zip)
    
    # Step 2: date-state combination for remaining NaNs
    if df_windchill_fill[windchill].isna().any():
        remaining = df_windchill_fill[windchill].isna().sum()
        print(f"Info: {remaining} wind chills still missing after date-zip fill.")
        print("Filling remaining with date-state mean.")
        
        windchill_means_state = df_windchill_fill.groupby([date, state])[windchill].transform('mean')
        df_windchill_fill[windchill] = df_windchill_fill[windchill].fillna(windchill_means_state)
    
    # Step 3: if any NaNs still remain, fill with overall mean as last resort
    if df_windchill_fill[windchill].isna().any():
        remaining = df_windchill_fill[windchill].isna().sum()
        print(f"Warning: {remaining} wind chills still missing after date-state fill.")
        print("Filling remaining with overall mean as last resort.")
        df_windchill_fill[windchill] = df_windchill_fill[windchill].fillna(df_windchill_fill[windchill].mean())
    
    return df_windchill_fill

df_windchill_fill = fill_missing_windchill(df_refined)

Info: 1949795 wind chills still missing after date-zip fill.
Filling remaining with date-state mean.
Filling remaining with overall mean as last resort.


In [23]:
# verifying NaNs have been replaced in the working df
df_windchill_fill['Wind_Chill(F)'].isna().sum()

0

In [24]:
# writing NaN-free column back to the original dataframe
df_refined['Wind_Chill(F)'] = df_windchill_fill['Wind_Chill(F)']
print(f"Entries with NaN Wind Chill: {df_refined['Wind_Chill(F)'].isna().sum()}")

Entries with NaN Wind Chill: 0


In [25]:
# deleting working df to free up memory
del df_windchill_fill
gc.collect()

112

In [26]:
# replacing NaN values in the humidity column

def fill_missing_hum(df, date='Acc_date', zip='Zipcode', state='State', hum='Humidity(%)'):
    """
    Replaces NaN humidity values with the mean humidity of entries
    with the same date and zip code. If no match exists with date and county,
    uses date and state.
    """
    # Step 1: mean humidity for each date-zip combination
    df_hum_fill = df_refined.copy()
    
    hum_means_zip = df_hum_fill.groupby([date, zip])[hum].transform('mean')
    
    # fill NaN values with the date-zip group mean
    df_hum_fill[hum] = df_hum_fill[hum].fillna(hum_means_zip)
    
    # Step 2: date-state combination for remaining NaNs
    if df_hum_fill[hum].isna().any():
        remaining = df_hum_fill[hum].isna().sum()
        print(f"Info: {remaining} humidity entries still missing after date-zip fill.")
        print("Filling remaining with date-state mean.")
        
        hum_means_state = df_hum_fill.groupby([date, state])[hum].transform('mean')
        df_hum_fill[hum] = df_hum_fill[hum].fillna(hum_means_state)
    
    # Step 3: if any NaNs still remain, fill with overall mean as last resort
    if df_hum_fill[hum].isna().any():
        remaining = df_hum_fill[hum].isna().sum()
        print(f"Warning: {remaining} humidity entries still missing after date-state fill.")
        print("Filling remaining with overall mean as last resort.")
        df_hum_fill[hum] = df_hum_fill[hum].fillna(df_hum_fill[hum].mean())
    
    return df_hum_fill

df_hum_fill = fill_missing_hum(df_refined)

Info: 166329 humidity entries still missing after date-zip fill.
Filling remaining with date-state mean.
Filling remaining with overall mean as last resort.


In [27]:
# verifying NaNs have been replaced in the working df
df_hum_fill['Humidity(%)'].isna().sum()

0

In [28]:
# writing NaN-free column back to the original dataframe
df_refined['Humidity(%)'] = df_hum_fill['Humidity(%)']
print(f"Entries with NaN humidity: {df_refined['Humidity(%)'].isna().sum()}")

Entries with NaN humidity: 0


In [29]:
# deleting working df to free up memory
del df_hum_fill
gc.collect()

48

In [33]:
# replacing NaN values in the pressure column

def fill_missing_press(df, date='Acc_date', zip='Zipcode', state='State', press='Pressure(in)'):
    """
    Replaces NaN pressure values with the mean pressure of entries
    with the same date and zip code. If no match exists with date and county,
    uses date and state.
    """
    # Step 1: mean pressure for each date-zip combination
    df_press_fill = df_refined.copy()
    
    press_means_zip = df_press_fill.groupby([date, zip])[press].transform('mean')
    
    # fill NaN values with the date-zip group mean
    df_press_fill[press] = df_press_fill[press].fillna(press_means_zip)
    
    # Step 2: date-state combination for remaining NaNs
    if df_press_fill[press].isna().any():
        remaining = df_press_fill[press].isna().sum()
        print(f"Info: {remaining} pressure entries still missing after date-zip fill.")
        print("Filling remaining with date-state mean.")
        
        press_means_state = df_press_fill.groupby([date, state])[press].transform('mean')
        df_press_fill[press] = df_hum_fill[press].fillna(press_means_state)
    
    # Step 3: if any NaNs still remain, fill with overall mean as last resort
    if df_press_fill[press].isna().any():
        remaining = df_press_fill[press].isna().sum()
        print(f"Warning: {remaining} pressure entries still missing after date-state fill.")
        print("Filling remaining with overall mean as last resort.")
        df_press_fill[press] = df_press_fill[press].fillna(df_press_fill[press].mean())
    
    return df_press_fill

df_press_fill = fill_missing_press(df_refined)

Info: 137561 pressure entries still missing after date-zip fill.
Filling remaining with date-state mean.
Filling remaining with overall mean as last resort.


In [34]:
# verifying NaNs have been replaced in the working df
df_press_fill['Pressure(in)'].isna().sum()

0

In [35]:
# writing NaN-free column back to the original dataframe
df_refined['Pressure(in)'] = df_press_fill['Pressure(in)']
print(f"Entries with NaN pressure: {df_refined['Pressure(in)'].isna().sum()}")

Entries with NaN pressure: 0


In [36]:
# deleting working df to free up memory
del df_press_fill
gc.collect()

48

In [37]:
# replacing NaN values in the visibility column

def fill_missing_vis(df, date='Acc_date', zip='Zipcode', state='State', vis='Visibility(mi)'):
    """
    Replaces NaN visibility values with the mean visibility of entries
    with the same date and zip code. If no match exists with date and county,
    uses date and state.
    """
    # Step 1: mean visibility for each date-zip combination
    df_vis_fill = df_refined.copy()
    
    vis_means_zip = df_vis_fill.groupby([date, zip])[vis].transform('mean')
    
    # fill NaN values with the date-zip group mean
    df_vis_fill[vis] = df_vis_fill[vis].fillna(vis_means_zip)
    
    # Step 2: date-state combination for remaining NaNs
    if df_vis_fill[vis].isna().any():
        remaining = df_vis_fill[vis].isna().sum()
        print(f"Info: {remaining} visibility entries still missing after date-zip fill.")
        print("Filling remaining with date-state mean.")
        
        vis_means_state = df_vis_fill.groupby([date, state])[vis].transform('mean')
        df_vis_fill[vis] = df_hum_fill[vis].fillna(vis_means_state)
    
    # Step 3: if any NaNs still remain, fill with overall mean as last resort
    if df_vis_fill[vis].isna().any():
        remaining = df_vis_fill[vis].isna().sum()
        print(f"Warning: {remaining} visibility entries still missing after date-state fill.")
        print("Filling remaining with overall mean as last resort.")
        df_vis_fill[vis] = df_vis_fill[vis].fillna(df_vis_fill[vis].mean())
    
    return df_vis_fill

df_vis_fill = fill_missing_vis(df_refined)

Info: 171980 visibility entries still missing after date-zip fill.
Filling remaining with date-state mean.
Filling remaining with overall mean as last resort.


In [38]:
# verifying NaNs have been replaced in the working df
df_vis_fill['Visibility(mi)'].isna().sum()

0

In [39]:
# writing NaN-free column back to the original dataframe
df_refined['Visibility(mi)'] = df_vis_fill['Visibility(mi)']
print(f"Entries with NaN visibility: {df_refined['Visibility(mi)'].isna().sum()}")

Entries with NaN visibility: 0


In [40]:
# deleting working df to free up memory
del df_vis_fill
gc.collect()

48

In [42]:
# replacing NaN with 0 in wind speed and precipitation

df_refined['Wind_Speed(mph)'] = df_refined['Wind_Speed(mph)'].fillna(0)
df_refined['Precipitation(in)'] = df_refined['Precipitation(in)'].fillna(0)
print(f"Entries with NaN Wind Speed: {df_refined['Wind_Speed(mph)'].isna().sum()}")
print(f"Entries with NaN Precipitation: {df_refined['Precipitation(in)'].isna().sum()}")

Entries with NaN Wind Speed: 0
Entries with NaN Precipitation: 0


In [ ]:
#print(f"Count of Unique Weather Condition Entries: {df_all_data['Weather_Condition'].nunique()}")
df_refined['Weather_Condition'].value_counts().sort_values(ascending=True)

In [ ]:
# printing a list of weather condition unique values to csv
unique_wthr_cond = pd.Series(df_refined['Weather_Condition'].unique())
unique_wthr_cond.to_csv('conditions.csv')

In [84]:
del df_test
del conditions_mapper
del mapper

In [85]:
mapper = {'Blowing Dust': 'Blowing Ash / Dust / Sand',
  'Blowing Dust / Windy': 'Blowing Ash / Dust / Sand',
  'Blowing Sand': 'Blowing Ash / Dust / Sand',
  'Blowing Snow': 'Blowing Snow',
  'Blowing Snow / Windy': 'Blowing Snow',
  'Blowing Snow Nearby': 'Blowing Snow',
  'Clear': 'Fair',
  'Cloudy': 'Cloudy',
  'Cloudy / Windy': 'Cloudy',
  'Drifting Snow': 'Blowing Snow',
  'Drifting Snow / Windy': 'Blowing Snow',
  'Drizzle': 'Light Rain',
  'Drizzle / Windy': 'Light Rain',
  'Drizzle and Fog': 'Light Rain',
  'Dust Whirls': 'Blowing Ash / Dust / Sand',
  'Duststorm': 'Blowing Ash / Dust / Sand',
  'Fair': 'Fair',
  'Fair / Windy': 'Fair',
  'Fog': 'Fog',
  'Fog / Windy': 'Fog',
  'Freezing Drizzle': 'Freezing Rain',
  'Freezing Rain': 'Freezing Rain',
  'Freezing Rain / Windy': 'Freezing Rain',
  'Funnel Cloud': 'Tornado',
  'Hail': 'Hail',
  'Haze': 'Hazy',
  'Haze / Windy': 'Hazy',
  'Heavy Blowing Snow': 'Heavy Snow',
  'Heavy Drizzle': 'Rain',
  'Heavy Freezing Drizzle': 'Freezing Rain',
  'Heavy Freezing Rain': 'Heavy Freezing Rain',
  'Heavy Freezing Rain / Windy': 'Heavy Freezing Rain',
  'Heavy Ice Pellets': 'Heavy Sleet',
  'Heavy Rain': 'Heavy Rain',
  'Heavy Rain / Windy': 'Heavy Rain',
  'Heavy Rain Shower': 'Heavy Rain',
  'Heavy Rain Shower / Windy': 'Heavy Rain',
  'Heavy Rain Showers': 'Heavy Rain',
  'Heavy Sleet': 'Heavy Sleet',
  'Heavy Sleet / Windy': 'Heavy Sleet',
  'Heavy Sleet and Thunder': 'Heavy Sleet',
  'Heavy Smoke': 'Heavy Smoke',
  'Heavy Snow': 'Heavy Snow',
  'Heavy Snow / Windy': 'Heavy Snow',
  'Heavy Snow with Thunder': 'Heavy Snow',
  'Heavy Thunderstorms and Rain': 'Heavy Thunderstorm',
  'Heavy Thunderstorms and Snow': 'Heavy Snow',
  'Heavy Thunderstorms with Small Hail': 'Heavy Thunderstorm',
  'Heavy T-Storm': 'Heavy Thunderstorm',
  'Heavy T-Storm / Windy': 'Heavy Thunderstorm',
  'Ice Pellets': 'Sleet',
  'Light Blowing Snow': 'Light Snow',
  'Light Drizzle': 'Light Rain',
  'Light Drizzle / Windy': 'Light Rain',
  'Light Fog': 'Light Fog',
  'Light Freezing Drizzle': 'Light Sleet',
  'Light Freezing Fog': 'Light Freezing Fog',
  'Light Freezing Rain': 'Light Sleet',
  'Light Freezing Rain / Windy': 'Light Sleet',
  'Light Hail': 'Light Hail',
  'Light Haze': 'Hazy',
  'Light Ice Pellets': 'Light Sleet',
  'Light Rain': 'Light Rain',
  'Light Rain / Windy': 'Light Rain',
  'Light Rain Shower': 'Light Rain',
  'Light Rain Shower / Windy': 'Light Rain',
  'Light Rain Showers': 'Light Rain',
  'Light Rain with Thunder': 'Light Rain',
  'Light Sleet': 'Light Sleet',
  'Light Sleet / Windy': 'Light Sleet',
  'Light Snow': 'Light Snow',
  'Light Snow / Windy': 'Light Snow',
  'Light Snow and Sleet': 'Wintry  Mix',
  'Light Snow and Sleet / Windy': 'Wintry  Mix',
  'Light Snow Grains': 'Light Snow',
  'Light Snow Shower': 'Light Snow',
  'Light Snow Shower / Windy': 'Light Snow',
  'Light Snow Showers': 'Light Snow',
  'Light Snow with Thunder': 'Light Snow',
  'Light Thunderstorm': 'Light Thunderstorm',
  'Light Thunderstorms and Rain': 'Light Thunderstorm',
  'Light Thunderstorms and Snow': 'Light Snow',
  'Low Drifting Snow': 'Light Snow',
  'Mist': 'Mist',
  'Mist / Windy': 'Mist',
  'Mostly Cloudy': 'Cloudy',
  'Mostly Cloudy / Windy': 'Cloudy',
  'N/A Precipitation': 'Fair',
  'Overcast': 'Cloudy',
  'Partial Fog': 'Light Fog',
  'Partial Fog / Windy': 'Light Fog',
  'Partly Cloudy': 'Partly Cloudy',
  'Partly Cloudy / Windy': 'Partly Cloudy',
  'Patches of Fog': 'Light Fog',
  'Patches of Fog / Windy': 'Light Fog',
  'Rain': 'Rain',
  'Rain / Windy': 'Rain',
  'Rain and Sleet': 'Sleet',
  'Rain Shower': 'Rain',
  'Rain Shower / Windy': 'Rain',
  'Rain Showers': 'Rain',
  'Sand': 'Blowing Ash / Dust / Sand',
  'Sand / Dust Whirls Nearby': 'Blowing Ash / Dust / Sand',
  'Sand / Dust Whirlwinds': 'Blowing Ash / Dust / Sand',
  'Sand / Dust Whirlwinds / Windy': 'Blowing Ash / Dust / Sand',
  'Sand / Windy': 'Blowing Ash / Dust / Sand',
  'Scattered Clouds': 'Partly Cloudy',
  'Shallow Fog': 'Light Fog',
  'Shallow Fog / Windy': 'Light Fog',
  'Showers in the Vicinity': 'Partly Cloudy',
  'Sleet': 'Sleet',
  'Sleet / Windy': 'Sleet',
  'Sleet and Thunder': 'Sleet',
  'Small Hail': 'Hail',
  'Smoke': 'Smoke',
  'Smoke / Windy': 'Smoke',
  'Snow': 'Snow',
  'Snow / Windy': 'Snow',
  'Snow and Sleet': 'Wintry  Mix',
  'Snow and Sleet / Windy': 'Wintry  Mix',
  'Snow and Thunder': 'Snow',
  'Snow and Thunder / Windy': 'Snow',
  'Snow Grains': 'Light Snow',
  'Snow Showers': 'Snow',
  'Squalls': 'Thunderstorm',
  'Squalls / Windy': 'Thunderstorm',
  'Thunder': 'Thunderstorm',
  'Thunder / Windy': 'Thunderstorm',
  'Thunder / Wintry Mix': 'Wintry  Mix',
  'Thunder / Wintry Mix / Windy': 'Wintry  Mix',
  'Thunder and Hail': 'Hail',
  'Thunder and Hail / Windy': 'Hail',
  'Thunder in the Vicinity': 'Light Thunderstorm',
  'Thunderstorm': 'Thunderstorm',
  'Thunderstorms and Rain': 'Thunderstorm',
  'Thunderstorms and Snow': 'Snow',
  'Tornado': 'Tornado',
  'T-Storm': 'Tornado',
  'T-Storm / Windy': 'Tornado',
  'Volcanic Ash': 'Blowing Ash / Dust / Sand',
  'Widespread Dust': 'Blowing Ash / Dust / Sand',
  'Widespread Dust / Windy': 'Blowing Ash / Dust / Sand',
  'Wintry Mix': 'Wintry  Mix',
  'Wintry Mix / Windy': 'Wintry  Mix'}

In [86]:
df_test = df_refined.copy()
df_test['Weather_Condition'] = df_test['Weather_Condition'].map(mapper)
print(df_test['Weather_Condition'].isna().sum)

MemoryError: Unable to allocate 767. MiB for an array with shape (13, 7728394) and data type object

In [83]:
df_test['Weather_Condition'].tail()

7728389    NaN
7728390    NaN
7728391    NaN
7728392    NaN
7728393    NaN
Name: Weather_Condition, dtype: object

## Analysis

Text here

## Evaluation

### Business Insight/Recommendation 1

### Business Insight/Recommendation 2

### Business Insight/Recommendation 3

### Tableau Dashboard link

## Conclusion and Next Steps
Text here